In [1]:
%pip install -r ~/code/GitHub/UPennContrast/devops/girder/annotation_client/requirements.txt
%pip install -e ~/code/GitHub/UPennContrast/devops/girder/annotation_client/
%pip install pandas
%pip install imageio
%pip install rtree
%pip install scikit-image
%pip install shapely
%pip install nd2reader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/arjunraj/code/GitHub/UPennContrast/devops/girder/annotation_client
  Attempting uninstall: annotation-client
    Found existing installation: annotation-client 0.1
    Uninstalling annotation-client-0.1:
      Successfully uninstalled annotation-client-0.1
  Running setup.py develop for annotation-client
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading 

In [6]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import pandas as pd
#import imageio
from shapely.geometry import Point, Polygon, LineString
#pd.set_option('display.max_colwidth', None)
import numpy as np

In [7]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

In [14]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

In [15]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

,Dataset Name
0,20200110_2Dent_001
1,20200110_2Dent_001_maxmerge
2,20210723_155544_798__Plate20210723_STR_gfp_Wel...
3,"20221122_143426_960__WellA4_ChannelDAPI,CY3,A5..."
4,"20230314_175616_594__WellB2_ChannelDAPI,YFP,CY..."
...,...
109,UI TEST OASL_SOX10_VGF_maxmerge
110,UInormmedia_8well_col2_livecellgfp
111,UInormmedia_8well_col2_livecellgfp_maxmerge
112,"WellA1_ChannelBrightfield,YFP,CY5,DAPI,A594_Se..."


In [16]:
dataset_name = 'SANDBOXnormmedia_8well_col2_livecellgfp'

In [17]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)
vars(datasetClient)
tile_width = datasetClient.tiles['tileWidth']
tile_height = datasetClient.tiles['tileHeight']
print(tile_width)
print(tile_height)

1024
1022


In [21]:
pointAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='point')
blobAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='polygon')

In [24]:
print(len(blobAnnotationList))
print(len(pointAnnotationList))
print(blobAnnotationList[0])
print(pointAnnotationList[0])

3
19
{'_id': '649b6f565d422edecab7fa26', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 258.03319195775197, 'y': 394.241097855824}, {'x': 257.08931742698894, 'y': 392.82528605967957}, {'x': 254.25769383469998, 'y': 389.2857565693183}, {'x': 253.07785067124624, 'y': 387.39800750779233}, {'x': 251.42607024241101, 'y': 386.45413297702936}, {'x': 250.0102584462665, 'y': 385.74622707895713}, {'x': 248.83041528281277, 'y': 384.8023525481941}, {'x': 246.9426662212868, 'y': 383.85847801743114}, {'x': 242.45926220016258, 'y': 383.38654075204965}, {'x': 240.80748177132733, 'y': 383.38654075204965}, {'x': 238.6837640771106, 'y': 384.3304152828126}, {'x': 237.50392091365688, 'y': 385.2742898135756}, {'x': 236.32407775020312, 'y': 385.9821957116479}, {'x': 234.90826595405863, 'y': 387.39800750779233}, {'x': 233.96439142329564, 'y': 388.81381930393684}, {'x': 233.49245415791415, 'y': 390.9375369981536}, {'x': 233.

Now let's try to push a new annotation

In [31]:
# Make a small square at the top left of the image
square_coords = [(10, 10),(30, 10),(30, 30),(10, 30)]

newAnnotation = {
    "tags": ['newTag'], # *** NEED TO UPDATE TO ADD A NEW TAG ****
    "shape": "polygon",
    "channel": blobAnnotationList[0]['channel'],
    "location": blobAnnotationList[0]['location'],
    "datasetId": blobAnnotationList[0]['datasetId'],
    "coordinates": [{"x": float(coord[0]), "y": float(coord[1])} for coord in square_coords]
}
annotationClient.createAnnotation(newAnnotation)


{'_id': '649b773a5d422edecab7fba9',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
 'channel': 1,
 'coordinates': [{'x': 10.0, 'y': 10.0},
  {'x': 30.0, 'y': 10.0},
  {'x': 30.0, 'y': 30.0},
  {'x': 10.0, 'y': 30.0}],
 'datasetId': '649b6efb5d422edecab7f9d8',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'polygon',
 'tags': ['newTag']}